In [1]:
# including the project directory to the notebook level
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import warnings

In [2]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    from keras import backend as K
    # from nn_source import models as mp
from dataprocess import dataprocessor as dp
import models as mp

Using TensorFlow backend.


In [3]:
"""This test checks that HyperOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import ray
from ray.tune import run
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
import argparse
from hyperopt import hp

In [9]:
# read the pickled file for ahu data
ahudata = dp.readfile('../data/processed/ahu1energy.pkl')

# return pickled df
ahu = ahudata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
ghidata = dp.readfile('../data/processed/ghi.pkl')

# return pickled df
ghi = ghidata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
flowdata = dp.readfile('../data/processed/flow.pkl')

# return pickled df
flow = flowdata.return_df(processmethods=['file2df'])
flow.columns = ['flow']

In [10]:
# selecting only required columns and rearranging them
ahu = ahu[[
    'AHU_1 outdoorAirTemp', 'WeatherDataProfile humidity',
    'AHU_1 supplyAirTemp', 'HW_BTU_METER currentKbtuDeltaReading',
    'CHW_BTU_METER currentKbtuDeltaReading'
]]

# renaming columns
ahu.columns = ['oat', 'orh', 'sat', 'hwe', 'cwe']

# Total energy is sum of heating and coling
ahu['totale'] = ahu['hwe'] + ahu['cwe']
# dropping heating and cooling energy columns
ahu = ahu.drop(columns=['hwe', 'cwe'])

In [12]:
# merging ahu and ghi data
df = dp.merge_df_columns([ahu, ghi, flow])

# rearranging columns
df = df[['oat', 'orh', 'sat', 'Ghi', 'flow', 'totale']]

In [4]:
# read the pickled file for ghi data
dfdata = dp.readfile('../data/processed/smoothbuildingdata.pkl')

# return pickled df
df = dfdata.return_df(processmethods=['file2df'])

In [5]:
df.head()

,oat,orh,sat,Ghi,flow,totale
2018-07-12 13:55:00,89.554909,76.0,75.443275,872.0,43.796002,37.726363
2018-07-12 14:00:00,89.863190,76.0,75.443275,866.0,43.780609,37.713771
2018-07-12 14:05:00,89.863190,76.0,75.443275,859.0,43.765213,37.701163
2018-07-12 14:10:00,90.484146,69.0,75.443275,848.0,43.749812,37.688538
2018-07-12 14:15:00,90.826981,69.0,75.443275,836.0,43.734408,37.675898


In [7]:
# Creating a list of 7 day dataframes for training
dflist = dp.df2dflist(df, subsequence=True, period=1, days=7, hours=0)

In [8]:
len(dflist)

10

In [9]:
# create list of training, testing arrays
weeklist = []
for weekdata in dflist:
    X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
        weekdata,
        predictorcols=['oat', 'orh', 'sat', 'Ghi', 'flow'],
        outputcols=['totale'],
        scaling=True,
        feature_range=(0,1),
        reshaping=True,
        lag=-1
    )
    weeklist.append({
        'X_train':X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
    })

In [10]:
weeklist[0].keys()
for key,value in weeklist[0].items():
    print("name: {}, value: {}".format(key, value.shape))

name: X_train, value: (1511, 1, 5)
name: y_train, value: (1511, 1, 1)
name: X_test, value: (504, 1, 5)
name: y_test, value: (504, 1, 1)


In [28]:
def objective_function3(config, reporter):
    
    modeldesigndone = False
    
    for weekdata in config['weeklist']:
        
        X_train = weekdata['X_train']
        y_train = weekdata['y_train']
        X_test = weekdata['X_test']
        y_test = weekdata['y_test']
    
        if not modeldesigndone:
            
            try:
                del model
            except NameError:
                pass

            K.clear_session()
            
            #Instantiate learner model
            model = mp.lstm_model(inputdim=X_train.shape[-1], outputdim = y_train.shape[-1], period = 1)

            # Desing model architecture
            model.design_model(lstmhiddenlayers=[config['lstm_hidden_units']]*config['lstm_no_layers'], 
                           densehiddenlayers=[config['dense_hidden_units']]*config['dense_no_layers'], 
                       dropoutlist=[[], []], batchnormalizelist=[[], []])
            
            # creating early stopping and learning reate changing callbacks
            model.model_callbacks()
            
            modeldesigndone = True
        
        # train the model
        model.train_model(X_train, y_train, X_test, y_test, epochs = 200)

        # evaluate the model for metrics at this stage
        cvrmse, mae = model.evaluate_model(X_train, y_train, X_test, y_test)

        # reporter for tracking performance of individual process/ ray process
        # note if it is reported betwen weekly data loop, then training might stop at the middle of the loop
        reporter(test_cvrmse=sum(cvrmse[1])/len(cvrmse[1]))
    

In [29]:
ray.init()

2020-03-11 15:12:39,434	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-11 15:12:39,437	INFO resource_spec.py:212 -- Starting Ray with 35.45 GiB memory available for workers and up to 17.75 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-11 15:12:40,068	INFO services.py:1078 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '129.59.104.221',
 'redis_address': '129.59.104.221:43558',
 'object_store_address': '/tmp/ray/session_2020-03-11_15-12-39_432522_18348/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-11_15-12-39_432522_18348/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-03-11_15-12-39_432522_18348'}

In [31]:
space = {
    'lstm_hidden_units': hp.choice("lstm_hidden_units", [1, 2, 4, 8, 16, 32, 64]),
    'lstm_no_layers': hp.choice("lstm_no_layers", [1, 2, 4, 8, 16]),
    'dense_hidden_units': hp.choice("dense_hidden_units", [1, 2, 4, 8, 16,]),
    'dense_no_layers' : hp.choice("dense_no_layers", [1, 2, 4, 8, 16, 32]),
}

config = {
    "num_samples": 150,
    "verbose":1,
    "resources_per_trial":{"cpu": 0.2},
    "config":{'weeklist':weeklist}
}

algo = HyperOptSearch(
    space,
    max_concurrent=40,
    metric="test_cvrmse",
    mode="min")


scheduler = AsyncHyperBandScheduler(
    metric="test_cvrmse",
    mode="min")

In [32]:
analysis = run(objective_function3, 
               search_alg=algo, 
               scheduler=scheduler, 
               **config)

Trial name,status,loc,lstm_no_layers,dense_hidden_units,lstm_hidden_units,dense_no_layers,total time (s),iter
objective_function3_e41c17c0,TERMINATED,,1,16,2,16,73.3082,1
objective_function3_d0f3a9ba,TERMINATED,,1,16,2,16,87.4354,1
objective_function3_c2bd8212,TERMINATED,,1,16,2,16,29.0869,1
objective_function3_bc20976e,TERMINATED,,1,16,2,16,94.9108,1
objective_function3_b639f78c,TERMINATED,,1,16,2,16,105.09,1
objective_function3_aa175e36,TERMINATED,,1,16,2,8,23.2897,1
objective_function3_a4236d08,TERMINATED,,16,1,32,2,258.815,1
objective_function3_9c7ec5c1,TERMINATED,,1,16,4,1,171.079,4
objective_function3_9c7ec5c0,TERMINATED,,2,8,16,32,145.81,1
objective_function3_9681abbc,TERMINATED,,8,16,2,1,281.646,1


2020-03-11 15:41:22,766	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


In [38]:
for i in space.keys():
    print("Best config for {} is".format(i), analysis.get_best_config(metric="test_cvrmse")[i])
    #    print("Best config for {} is".format(i), analysis.get_best_config(metric="test_cvrmse",mode='min')[i])

Best config for lstm_hidden_units is 4
Best config for lstm_no_layers is 2
Best config for dense_hidden_units is 8
Best config for dense_no_layers is 32


In [36]:
ray.shutdown()

In [37]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir ~/ray_results/objective_function3/

In [ ]:
%tensorboard --logdir ~/ray_results/objective_function/